# WHILE - Assignment 2
Robert Sato
rssato
1517254
1/22/2021

To Do:
- tokenize the input
- convert arith code to take list of tokens as input
- implement boolean expressions
- read "straight line code"
    - variables and assignment
- implement if
- implement while

Completed:
- tokenize the input
- convert arith code to take list of tokens as input

In [1]:
INTEGER, PLUS, MINUS, MUL, LPAREN, RPAREN, LBRACE, RBRACE, ASSIGNMENT, VAR, TRUE, FALSE, SEMI_COLON, EOF= (
    'INTEGER', 'PLUS', 'MINUS', 'MUL', '(', ')', '{', '}', 'ASSIGNMENT', 'VAR', 'TRUE', 'FALSE', ';', 'EOF'
)
IF, THEN, ELSE, WHILE, DO, SKIP, NOT, LESS_THAN, EQUALS, OR, AND = (
    'IF', 'THEN', 'ELSE', 'WHILE', 'DO', 'SKIP', 'NOT', '<', '==', 'OR', 'AND'
)

# token class for representing all the different types of symbols
class Token(object):
    def __init__(self, type, value):
        self.type = type
        self.value = value
    def __repr__(self):
        return 'Token({type}, {value})'.format(
            type=self.type,
            value=repr(self.value)
        )

In [2]:
# "tokenize" the input string - convert the input text into a list of tokens
# accepts numbers, parenthesis, and arithmetic symbols
# Given: no decimal numbers and always 1 white space

class Lexer(object):
    def __init__(self, text):
        self.text = text
        self.pos = 0
        self.tokenList = self.tokenize()
        self.token_pos = 0
    
    # return a list of tokens
    def tokenize(self):
        textAsTokens = []
        print("tokenize()\nEquation: ", self.text)
        #print("Text has len: ", len(self.text))
        while self.pos < len(self.text):
            curr = self.text[self.pos]
            #print('curr =', curr)
            # setup conditionals for all cases
            if curr == ' ':
                self.pos += 1
                continue
            elif curr.isdigit():
                # keep adding digit chars until whitespace
                num = ''
                while curr.isdigit(): # keep appending
                    num += curr
                    self.pos += 1
                    if self.pos >= len(self.text):
                        break
                    curr = self.text[self.pos]
                self.pos -= 1
                # convert string of digits to int
                num = int(num)
                textAsTokens.append(Token(INTEGER, num))
            elif curr == '+':
                textAsTokens.append(Token(PLUS, curr))
            elif curr == '*':
                textAsTokens.append(Token(MUL, curr))
            elif curr == '-':
                # add functionality for negative numbers
                # if next char is a digit, do digit stuff and add as INTEGER
                next = self.text[self.pos+1]
                if next.isdigit():
                    print('found next is digit:', next)
                    # do digit stuff
                    self.pos += 1
                    curr = self.text[self.pos]
                    num = '-'
                    while curr.isdigit(): # keep appending
                        num += curr
                        self.pos += 1
                        if self.pos >= len(self.text):
                            break
                        curr = self.text[self.pos]
                    self.pos -= 1
                    # convert string of digits to int
                    num = int(num)
                    textAsTokens.append(Token(INTEGER, num))

                # elif a space, add as MINUS
                else:
                #elif next == ' ':
                    textAsTokens.append(Token(MINUS, curr))
                # elif a LPAREN -> ur screwed so ignore for now
            elif curr == '(':
                textAsTokens.append(Token(LPAREN, curr))
            elif curr == ')':
                textAsTokens.append(Token(RPAREN, curr))
            elif curr == '{':
                textAsTokens.append(Token(LBRACE, curr))
            elif curr == '}':
                textAsTokens.append(Token(RBRACE, curr))
            elif curr == '¬':
                textAsTokens.append(Token(NOT, curr))
            elif curr == '<':
                textAsTokens.append(Token(LESS_THAN, curr))
            elif curr == '=':
                textAsTokens.append(Token(EQUALS, curr))
            elif curr == '∨':
                textAsTokens.append(Token(OR, curr))
            elif curr == '∧':
                textAsTokens.append(Token(AND, curr))
            elif curr == ';':
                textAsTokens.append(Token(SEMI_COLON, curr))
            # assignment ':='
            elif curr == ':':
                next = self.text[self.pos+1]
                if next == '=':
                    textAsTokens.append(Token(ASSIGNMENT, ':='))
                    self.pos += 1
                else:
                    print("Error: no = following : in expected :=")
            
            # keywords
            # if
            elif curr == 'i' and (self.pos < len(self.text) - 1 and self.text[self.pos+1] == 'f'):
                # skip one and add if token
                self.pos += 1
                textAsTokens.append(Token(IF, 'if'))
            # then
            elif curr == 't' and (self.pos < len(self.text) - 3 and self.text[self.pos+1:self.pos+4] == 'hen'):
                self.pos += 3
                textAsTokens.append(Token(THEN, 'then'))

            # else
            elif curr == 'e' and (self.pos < len(self.text) - 3 and self.text[self.pos+1:self.pos+4] == 'lse'):
                self.pos += 3
                textAsTokens.append(Token(ELSE, 'else'))

            # while
            elif curr == 'w' and (self.pos < len(self.text) - 4 and self.text[self.pos+1:self.pos+5] == 'hile'):
                self.pos += 4
                textAsTokens.append(Token(WHILE, 'while'))

            # do
            elif curr == 'd' and (self.pos < len(self.text) - 1 and self.text[self.pos+1] == 'o'):
                self.pos += 1
                textAsTokens.append(Token(DO, 'do'))

            # skip
            elif curr == 's' and (self.pos < len(self.text) - 3 and self.text[self.pos+1:self.pos+4] == 'kip'):
                self.pos += 3
                textAsTokens.append(Token(SKIP, 'skip'))
            
            # true
            elif curr == 't' and (self.pos < len(self.text) - 3 and self.text[self.pos+1:self.pos+4] == 'rue'):
                self.pos += 3
                textAsTokens.append(Token(TRUE, 'true'))
            
            # false
            elif curr == 'f' and (self.pos < len(self.text) - 4 and self.text[self.pos+1:self.pos+5] == 'alse'):
                self.pos += 4
                textAsTokens.append(Token(FALSE, 'false'))

            # variable - check this after checking if 'true' 'false' 'if' etc.
            elif curr.isalpha():
                # all following characters except whitespace counts as the variable name
                var = ''
                while curr != ' ': # keep appending
                    var += curr
                    self.pos += 1
                    if self.pos >= len(self.text):
                        break
                    curr = self.text[self.pos]
                self.pos -= 1
                # convert string of digits to int
                textAsTokens.append(Token(VAR, var))

            else:
                print("Error found unknown symbol at '{}'".format(curr))
            
            self.pos += 1

        return textAsTokens
    
    # for the parser library
    def get_next_token(self):
        # can only be called after calling tokenize
        #print("Token list is size:", len(self.tokenList))
        if self.token_pos < len(self.tokenList):
            temp = self.tokenList[self.token_pos]
            self.token_pos += 1
            print("get_next_token() =", temp)
            return temp
        else:
            #print("No more tokens")
            return Token(EOF, None)

    def print(self):
        print("Text has len: ", len(self.text))
        while self.pos < len(self.text):
            print("{}:{}".format(self.pos, self.text[self.pos]))
            self.pos += 1

    def getNextChar(self):
        self.pos += 1
        return self.text[self.pos-1]

In [21]:
lexer = Lexer('true ∨ false')
parser = Parser(lexer)

tokenize()
Equation:  true ∨ false
get_next_token() = Token(TRUE, 'true')


In [3]:
formulas = []
formulas.append('x := 1')
formulas.append('x := (1 + 2) * 3; y := 3')
formulas.append('if 0 < x ∨ 4 < y then x := 1 else x := 3')

formulas.append('a := 98 ; b := 76 ; while ¬ ( a = b ) do { if a < b then b := b - a else a := a - b }')
formulas.append('while false do x := 1 ; y := 1')

formulas.append('if 3 < -3 then g := 3 + -2 else h := 09 + 9')
formulas.append('if ( false ∨ 3 < y + X ) then l := lv + -1 else x := -4 - z ; while -1 - p = 2 - -3 ∧ false do while ( ¬ ( 2 * -2 < y * y ) ) do skip')

for formula in formulas:
    lexer = Lexer(formula)
    token = lexer.get_next_token()
    while token.type != EOF:
        #print(token)
        token = lexer.get_next_token()
    print()

tokenize()
Equation:  x := 1
get_next_token() = Token(VAR, 'x')
get_next_token() = Token(ASSIGNMENT, ':=')
get_next_token() = Token(INTEGER, 1)

tokenize()
Equation:  x := (1 + 2) * 3; y := 3
get_next_token() = Token(VAR, 'x')
get_next_token() = Token(ASSIGNMENT, ':=')
get_next_token() = Token((, '(')
get_next_token() = Token(INTEGER, 1)
get_next_token() = Token(PLUS, '+')
get_next_token() = Token(INTEGER, 2)
get_next_token() = Token(), ')')
get_next_token() = Token(MUL, '*')
get_next_token() = Token(INTEGER, 3)
get_next_token() = Token(;, ';')
get_next_token() = Token(VAR, 'y')
get_next_token() = Token(ASSIGNMENT, ':=')
get_next_token() = Token(INTEGER, 3)

tokenize()
Equation:  if 0 < x ∨ 4 < y then x := 1 else x := 3
get_next_token() = Token(IF, 'if')
get_next_token() = Token(INTEGER, 0)
get_next_token() = Token(<, '<')
get_next_token() = Token(VAR, 'x')
get_next_token() = Token(OR, '∨')
get_next_token() = Token(INTEGER, 4)
get_next_token() = Token(<, '<')
get_next_token() = Token(V

In [104]:
# create an AST from the tokens
class AST(object):
    pass

class BinOp(AST):
    def __init__(self, left, op, right):
        self.left = left
        self.token = self.op = op
        self.right = right

class Num(AST):
    def __init__(self, token):
        self.token = token
        self.value = token.value
    def __repr__(self):
        return 'Num({value})'.format(
            value=repr(self.value)
        )

class Var(AST):
    def __init__(self, token):
        self.token = token
        self.value = token.value
    def __repr__(self):
        return 'Var({value})'.format(
            value=repr(self.value)
        )

In [105]:
'''
EBNF Grammer from for boolean expressions: https://unnikked.ga/how-to-build-a-boolean-expression-evaluator-518e9e068a65
<expression>::=<term>{<or><term>}
<term>::=<factor>{<and><factor>}
<factor>::=<constant>|<not><factor>|(<expression>)
<constant>::= false|true
<or>::='|'
<and>::='&'
<not>::='!'
'''

"\nEBNF Grammer from for boolean expressions: https://unnikked.ga/how-to-build-a-boolean-expression-evaluator-518e9e068a65\n<expression>::=<term>{<or><term>}\n<term>::=<factor>{<and><factor>}\n<factor>::=<constant>|<not><factor>|(<expression>)\n<constant>::= false|true\n<or>::='|'\n<and>::='&'\n<not>::='!'\n"

In [119]:
# node types for boolean expressions
class BoolOp(AST):
    # handles cases for multiple boolean expressions
    def __init__(self, left, op, right):
        self.left = left
        # if expr comparison, the operation will be = or <
        # else it will be AND or OR
        self.token = self.op = op
        self.right = right

class BoolLeaf(AST):
    # handles single bool expr cases
    def __init__(self, token):
        self.token = token
        self.value = token.value
    def __repr__(self):
        return 'BoolLeaf({value})'.format(
            value=repr(self.value)
        )

class Not(AST):
    def __init__(self, child):
        self.child = child
    def __repr__(self):
        return 'Not({child})'.format(
            child=self.child
        )

In [128]:
# Parser code from https://ruslanspivak.com/lsbasi-part7/ in references;
# changed for my list implementation of tokens
class Parser(object):
    def __init__(self, lexer):
        self.lexer = lexer
        # set current token to the first token taken from the input
        self.current_token = self.lexer.get_next_token()

    def error(self):
        raise Exception('Invalid syntax')

    def eat(self, token_type):
        # compare the current token type with the passed token
        # type and if they match then "eat" the current token
        # and assign the next token to the self.current_token,
        # otherwise raise an exception.
        print("Eating: type: {}, value: {}".format(self.current_token.type, self.current_token.value))
        if self.current_token.type == token_type:
            self.current_token = self.lexer.get_next_token()
        else:
            self.error()

    def factor(self):
        """factor : INTEGER | LPAREN expr RPAREN"""
        token = self.current_token
        if token.type == INTEGER:
            self.eat(INTEGER)
            return Num(token)
        elif token.type == VAR:
            self.eat(VAR)
            return Var(token)
        elif token.type == LPAREN:
            self.eat(LPAREN)
            node = self.expr()
            self.eat(RPAREN)
            return node

    def term(self):
        """term : factor ((MUL) factor)*"""
        node = self.factor()

        while self.current_token.type in (MUL):
            token = self.current_token
            if token.type == MUL:
                self.eat(MUL)

            node = BinOp(left=node, op=token, right=self.factor())

        return node

    def expr(self):
        """
        expr   : term ((PLUS | MINUS) term)*
        term   : factor ((MUL | DIV) factor)*
        factor : INTEGER | LPAREN expr RPAREN
        """
        node = self.term()

        while self.current_token.type in (PLUS, MINUS):
            token = self.current_token
            if token.type == PLUS:
                self.eat(PLUS)
            elif token.type == MINUS:
                self.eat(MINUS)

            node = BinOp(left=node, op=token, right=self.term())

        return node


    # define a boolean expression type
    def b_expr(self):
        """
        <expression>::=<term>{<or><term>}
        """
        print('in b_expr()')
        print('current_token:', self.current_token)
        # first token in set (true, false, expr, not, b_expr)
        node = self.b_term()
        #print('self.current_token after b_expr call to b_term():', self.current_token)
        while self.current_token.type == OR:
            token = self.current_token
            self.eat(OR)
            node = BoolOp(left=node, op=token, right=self.b_term())
        return node

    def b_term(self):
        """
        <term>::=<factor>{<and><factor>}
        """
        print('in b_term()')
        print('current_token:', self.current_token)
        node = self.b_factor()
        while self.current_token.type == AND:
            token = self.current_token
            self.eat(AND)
            node = BoolOp(left=node, op=token, right=self.b_factor())
        return node
    
    def b_factor(self):
        """
        <factor>::=<constant>|<not><factor>|(<expression>)
        <constant>::= false|true
        """
        print('in b_factor()')
        print('current_token:', self.current_token)
        token = self.current_token
        if token.type == TRUE:
            self.eat(TRUE)
            return BoolLeaf(token)
        elif token.type == FALSE:
            self.eat(FALSE)
            return BoolLeaf(token)
        elif token.type == NOT:
            self.eat(NOT)
            node = self.b_factor()
            node = Not(child=node)
            return node
        elif token.type == LPAREN:
            self.eat(LPAREN)
            node = self.b_expr()
            self.eat(RPAREN)
            return node
        else: # comparisons
            #print("Error handling b_factor()")
            node = self.expr()
            token = self.current_token
            self.eat(token.type)
            #print("Comparison token is:", token)
            node = BoolOp(left=node, op=token, right=self.expr())
            return node

    # for commands
    def c_expr(self):
        # input is a list of commands
        while self.current_token.type in (SKIP, SEMI_COLON, VAR, IF, WHILE):
            if self.current_token.type == SKIP:
                # eat the skip token and semicolon if exists
                print("skip does nothing!")
                self.eat(SKIP)
            elif self.current_token.type == SEMI_COLON:
                self.eat(SEMI_COLON)
            elif self.current_token.type == VAR:
                # create a node that will create a var and assign the value of an arith expr to it
                # should call self.expr()
                print("found variable assignment for:", self.current_token.value)
                self.eat(VAR)
                self.eat(ASSIGNMENT)
                expr_node = self.expr()
                print("expr returned:")
                printAST(expr_node)
            elif self.current_token.type == IF:
                print("idk how to deal with if yet!!")
                self.eat(IF)
            elif self.current_token.type == WHILE:
                print("idk how to deal with while yet!!")
                self.eat(WHILE)

    def parse(self):
        #return self.expr()
        return self.b_expr() # testing booleans
        #return self.c_expr()

In [140]:
def printAST(node):
    print("printAST() type:", type(node))
    if type(node) == BinOp:
        printAST(node.left)
        print(node.op)
        printAST(node.right)
    elif type(node) == BoolOp:
        printAST(node.left)
        print(node.op)
        printAST(node.right)
    elif type(node) == BoolLeaf:
        print(node)
    elif type(node) == Not:
        printAST(node.child)
    else:
        print(node)

In [141]:
boolexprs = []
boolexprs.append('true ∨ false ∨ false')
boolexprs.append('¬ true')
boolexprs.append('-1 < -2')
boolexprs.append('x < 3')
boolexprs.append('( true ∧ true )')
boolexprs.append('( ¬ ( 0 - -1 < 2 + z ) )')
for boolexpr in boolexprs:
    lexer = Lexer(boolexpr)
    parser = Parser(lexer)
    node = parser.parse()
    print("Done parsing!\n")
    #print(type(node))
    printAST(node)
    print()


tokenize()
Equation:  true ∨ false ∨ false
get_next_token() = Token(TRUE, 'true')
in b_expr()
current_token: Token(TRUE, 'true')
in b_term()
current_token: Token(TRUE, 'true')
in b_factor()
current_token: Token(TRUE, 'true')
Eating: type: TRUE, value: true
get_next_token() = Token(OR, '∨')
Eating: type: OR, value: ∨
get_next_token() = Token(FALSE, 'false')
in b_term()
current_token: Token(FALSE, 'false')
in b_factor()
current_token: Token(FALSE, 'false')
Eating: type: FALSE, value: false
get_next_token() = Token(OR, '∨')
Eating: type: OR, value: ∨
get_next_token() = Token(FALSE, 'false')
in b_term()
current_token: Token(FALSE, 'false')
in b_factor()
current_token: Token(FALSE, 'false')
Eating: type: FALSE, value: false
Done parsing!

printAST() type: <class '__main__.BoolOp'>
printAST() type: <class '__main__.BoolOp'>
printAST() type: <class '__main__.BoolLeaf'>
BoolLeaf('true')
Token(OR, '∨')
printAST() type: <class '__main__.BoolLeaf'>
BoolLeaf('false')
Token(OR, '∨')
printAST() type

In [22]:
# create an interpretor that does the computation given the AST
class Interpretor(object):
    def __init__(self, rootAST):
        self.rootAST = rootAST

    def interpret(self):
        return self.visit(self.rootAST)

    def visit(self, node):
        # call operation for appropriate BinOp or just Number type
        node_type = type(node).__name__
        if node_type ==  'BinOp':
            if node.op.type == PLUS:
                return self.visit(node.left) + self.visit(node.right)
            elif node.op.type == MINUS:
                return self.visit(node.left) - self.visit(node.right)
            elif node.op.type == MUL:
                return self.visit(node.left) * self.visit(node.right)
        else:
            return node.value

In [23]:
for formula in formulas:
    lexer = Lexer(formula)
    parser = Parser(lexer)
    node = parser.parse()
    interpretor = Interpretor(node)
    val = interpretor.interpret()
    print("=", val)

tokenize()
Equation:  1 + 2 * 3
curr = 1
curr =  
curr = +
curr =  
curr = 2
curr =  
curr = *
curr =  
curr = 3
= 7
tokenize()
Equation:  (1 + 2) * 3
curr = (
curr = 1
curr =  
curr = +
curr =  
curr = 2
curr = )
curr =  
curr = *
curr =  
curr = 3
= 9
tokenize()
Equation:  ((1+ 2)) * 3
curr = (
curr = (
curr = 1
curr = +
curr =  
curr = 2
curr = )
curr = )
curr =  
curr = *
curr =  
curr = 3
= 9
tokenize()
Equation:  15 * (25 + 10)
curr = 1
curr =  
curr = *
curr =  
curr = (
curr = 2
curr =  
curr = +
curr =  
curr = 1
curr = )
= 525
tokenize()
Equation:  -5 + 10 * (3 * -3)
curr = -
found next is digit: 5
curr =  
curr = +
curr =  
curr = 1
curr =  
curr = *
curr =  
curr = (
curr = 3
curr =  
curr = *
curr =  
curr = -
found next is digit: 3
curr = )
= -95


# Next Steps:
- test on test script
- implement negative numbers DONE